In [ ]:
import os
import pandas as pd
import sys

from pathlib import Path

# Custom imports
sys.path.append(os.path.join(
    os.path.abspath('.'), 'notebooks'))

from commons import *

In [ ]:
FILE_BBRef_players = 'BBRef_Ids_to_Players.csv'
FILE_BBRef_teams   = 'BBRef_Ids_to_Teams.csv'

In [ ]:
df_BBRef_players = pd.read_csv(DIR_raw / FILE_BBRef_players)
df_BBRef_teams   = pd.read_csv(DIR_raw / FILE_BBRef_teams)

## Datasets processing

### Ids to Players

In [ ]:
df_BBRef_players = df_BBRef_players[['BBRefName', 'BBRefID', 'BBRefBirthDate']].dropna()

df_BBRef_players['BBRefID']   = df_BBRef_players['BBRefID'].str.capitalize()
df_BBRef_players['BBRefName'] = df_BBRef_players['BBRefName'].apply(lambda s: unicd2ascii(s))

df_BBRef_players.rename(columns={'BBRefName': 'Name',
                                 'BBRefID': 'Id',
                                 'BBRefBirthDate': 'DOB'}, inplace=True)

df_BBRef_players.set_index('Id', inplace=True)
df_BBRef_players.to_csv(DIR_raw / 'Ids_to_Players.csv')

### Ids to Teams 

In [ ]:
df_BBRef_teams   = df_BBRef_teams[['BBRef_Team_Name', 'BBRef_Team_Abbreviation']].dropna()
df_BBRef_teams   = df_BBRef_teams.drop_duplicates()

df_BBRef_teams.rename(columns={'BBRef_Team_Name': 'Name',
                               'BBRef_Team_Abbreviation': 'Id'}, inplace=True)

df_BBRef_teams.set_index('Id', inplace=True)
df_BBRef_teams.to_csv(DIR_raw / 'Ids_to_Teams.csv')

### Clutch time all NBA stats 

In [ ]:
for y in range(YEAR_start, YEAR_end + 1):
    filename = year2filename(y)
    
    df_pace   = pd.read_csv(DIR_raw_pace / filename)
    df_teams  = pd.read_csv(DIR_raw_teams / filename)
    df_teams  = pd.merge(df_teams, df_pace, on='Team')
    
    df_league = pd.DataFrame(df_teams.drop('Team', 1).mean())
    df_league = df_league.T
    
    df_league.to_csv(DIR_processed_league
                     / filename, index=False)